In [1]:
import numpy as np
import sympy as sp
from plotly import graph_objects as go
import script as rt

Прво си измислуваме / дефинираме податоците.

In [2]:
h_slam_points = np.array([[1, 2, 1], [3, 5, 1], [4, 7, 1], [5, 8, 1]])
transform = rt.hrotation2(np.pi/3) * rt.htranslation2(2.7, 3)
transform = np.array(transform, dtype=np.float)
h_sensor_points = (transform @ h_slam_points.T).T

Изврши ја прво оваа ќелија, потоа наредната и гледај го исходот на графикот.

In [3]:
fig = go.FigureWidget()
fig.update_layout(xaxis={'range': [-7, 6]}, yaxis={'range': [1, 13]}),
fig.add_trace(go.Scatter(x=h_sensor_points[:, 0], y=h_sensor_points[:, 1], mode='markers', name='Сензори',
                         marker={'size': 10, 'symbol': 'x', 'color': np.arange(len(h_sensor_points))}))
fig.add_trace(go.Scatter(x=h_slam_points[:, 0], y=h_slam_points[:, 1], mode='markers', name='SLAM', 
                         marker={'size': 10, 'color': np.arange(len(h_slam_points))}))

fig

FigureWidget({
    'data': [{'marker': {'color': array([0, 1, 2, 3]), 'size': 10, 'symbol': 'x'},
            …

In [6]:
def error(h_sensor_points, h_slam_points, x, y, alpha):
    """
    Функција која ја пресметува грешката, на начин на кој сме ја дефинирале.
    """
    transform = rt.hrotation2(alpha) * rt.htranslation2(x, y)
    transform = np.array(transform, dtype=np.float)
    h_hat_sensor_points = (transform @ h_slam_points.T).T
    e = h_sensor_points - h_hat_sensor_points
    return np.apply_along_axis(lambda x: x.T @ x, 1, e)

def step_gradient(h_sensor_points, h_slam_points, transform, learning_rate):
    """
    Чекор со gradient descent.
    
    h_sensor_points, h_slam_points: се податоците кои ќе ни послужат за пресметка на грешката.
    transform: е матрицата на трансформација чии членови ги бараме во оваа функција
    learning_rate: стапката на учење
    """
    x_gradient, y_gradient, alpha_gradient = 0, 0, 0
    h_hat_sensor_points = (transform @ h_slam_points.T).T  # чекорот го почнуваме од моменталното предвидување...
    r_vector = h_sensor_points - h_hat_sensor_points  # ...сѐ со цел да ја пресметаме грешката која ќе ни треба понатаму
    r_x_vector, r_y_vector = r_vector[:, 0], r_vector[:, 1]  # тука ги вадиме грешките по x и y за секоја точка. Користиме хомогени координати
    m = len(r_vector)
    for r_x, r_y, h_sensor_point in zip(r_x_vector, r_y_vector, h_sensor_points):
        x_gradient += r_x  # го пресметуваме градиентот по x
        y_gradient += r_y  # го пресметуваме градиентот по y
        alpha_gradient += -h_sensor_point[1] * r_x + h_sensor_point[0] * r_y    # го пресметуваме градиентот за аголот
    dx = -learning_rate * x_gradient / m  # потоа со градиентот откриваме колкав чекор треба да направиме по x
    dy = -learning_rate * y_gradient / m  # потоа со градиентот откриваме колкав чекор треба да направиме по y
    dalpha = -learning_rate/10 * alpha_gradient / m   # потоа со градиентот откриваме колкав чекор треба да направиме по alpha
    # претходните 3 чекори по секој степен на слобода ги соединуваме во една трансформациона матрица
    delta_transform = np.array(rt.hrotation2(dalpha) * rt.htranslation2(dx, dy), dtype=np.float)
    # и конечно го правиме чекорот
    transform = delta_transform @ transform
    # на крајот ја враќаме обновената вредност на transform и предвидената вредност за изместените податоци за да ги исцртаме
    return transform, h_hat_sensor_points

def run(fig, h_sensor_points, h_slam_points):
    learning_rate = -0.01
    num_iterations = 10000
    # почнуваме со случајно избрани вредности за x, y, и alpha
    x, y, alpha = 0, 0, 0
    transform = np.array(rt.hrotation2(alpha) * rt.htranslation2(x, y), dtype=np.float)

    for i in range(num_iterations):
        transform, h_hat_sensor_points = step_gradient(h_sensor_points, h_slam_points, transform, learning_rate)
        # цртаме на секоја стота итерација
        if i % 100 == 0:
            x, y, alpha = transform[0, 2], transform[1, 2], np.arctan2(transform[1, 0], transform[0, 0]) / np.pi * 180
            with fig.batch_update():
                fig.data[1].x, fig.data[1].y = h_hat_sensor_points[:, 0], h_hat_sensor_points[:, 1]
                fig.data[1].marker.color = np.arange(len(h_hat_sensor_points))
    
    x, y, alpha = transform[0, 2], transform[1, 2], np.arctan2(transform[1, 0], transform[0, 0]) / np.pi * 180
    return x, y, alpha

run(fig, h_sensor_points, h_slam_points)

(-1.2515391163784921, 3.836349552050522, 59.96780574578148)